In [ ]:

import sys
from pathlib import Path
import seaborn as sns

In [ ]:

LCL_PATH  = str(Path().cwd().parent.parent)
ROOT_PATH = str(Path(LCL_PATH).parent.parent)
print("""
root path:\t{}
local path:\t{}""".format(ROOT_PATH, LCL_PATH))

In [ ]:

# adding local_folder to the system path
sys.path.append(LCL_PATH)
sys.path.append(ROOT_PATH)

from sys_function import * # este in root

In [ ]:

sys_remove_modules("imshow")
sys_remove_modules("builds.init_population.ant_colony")
sys_remove_modules("extension.utils.normalization")
sys_remove_modules("builds.dataset_man.dataset_ttp_man")
sys_remove_modules("builds.metrics.metrics_ttp_base")

from imshow import *
from builds.init_population.ant_colony import *
from extension.utils.normalization import *
from builds.dataset_man.dataset_ttp_man import *
from builds.metrics.metrics_ttp_base import *

# Data aquisition

In [ ]:

dataset_man_obj = DatasetTTPMan("{}/data".format(LCL_PATH))
dataset = dataset_man_obj(nodes_file="NODE_COORD_SECTION.csv",
                        items_file="ITEMS_SECTION.csv")
dataset

In [ ]:
dataset["distance"][169:172]

In [ ]:
metrics_obj = MetricsTTPBase(dataset, 0)

In [ ]:

filename_population = "{}/ant_routes.npy".format(LCL_PATH)
with open(filename_population, 'wb') as f:
    np.save(f, np.array(tmp_routes))

In [ ]:

filename_population = "{}/ant_routes.npy".format(LCL_PATH)
with open(filename_population, 'rb') as f:
    ant_routes = np.load(f)

# Ant Colony Optimization

In [ ]:

ant_colony_obj = AntColonyInitTTP(metrics=metrics_obj, alpha=1.0, beta=11.0, rho=0.5, q=1.0)

In [ ]:
ant_colony_obj.updateByElite(ant_routes)

In [ ]:

new_route = np.arange(ant_colony_obj.GENOME_LENGTH, dtype=np.int32)
old_route = new_route
ant_colony_obj.updateByElite([new_route])

In [ ]:
old_route = new_route
tmp_routes = []
best_route = None
best_cost  = np.inf

In [ ]:

for i in range(500):
    print("Start noul inceput {} ___________".format(i))
    ant_colony_obj.updateByElite(np.array(tmp_routes), q=0.3, rho=0.9)
    new_route = ant_colony_obj(start_city=0, generations=200, size_ants=35, monitor_size=25)
    tmp_routes.append(new_route)
    cost  = metrics_obj.computeIndividDistance(new_route)
    if (cost < best_cost):
        best_cost  = cost
        best_route = new_route
    print("Stop noul inceput {} ============".format(i))

In [ ]:

tau_image = (normalization(ant_colony_obj.tau) * 255).astype(np.uint8)
image_show_wait(tau_image, window_name="Tau")

In [ ]:
np.argwhere(tau_image > 0.5)

In [ ]:
ant_colony_obj.best_path[0]

In [ ]:
dataset_man_obj.coords[ant_colony_obj.best_path[0]]

In [ ]:

map_all = dataset_man_obj.putRoutesOnMap(np.argwhere(tau_image > 100))
map_ant_best = dataset_man_obj.putRouteOnMap(ant_colony_obj.best_path)
new_distance = metrics_obj.computeIndividDistance(ant_colony_obj.best_path)
print("distance: {}".format(new_distance))
image_show(map_ant_best, window_name="Best")
image_show_wait(map_all, window_name="All")

In [ ]:
len(tmp_routes)

In [ ]:

old_distance = metrics_obj.computeIndividDistance(old_route)
new_distance = metrics_obj.computeIndividDistance(best_route)
print("old distance: {}, new distance: {}".format(old_distance, new_distance))
map_imgo = dataset_man_obj.putRouteOnMap(old_route)
map_imgp = dataset_man_obj.putRouteOnMap(best_route)
image_show(map_imgo, window_name="Old")
image_show_wait(map_imgp, window_name="AntColony")

In [ ]:

x = np.linspace(start=1, stop=0., num=100)
#data = {"x":x, "x_sqrt":np.sqrt(x), "x_log":np.log(x), "x_exp":np.exp(x)}
data = {"x":x, "x_sqrt":np.sqrt(x), "x_cbrt":np.cbrt(x), "x_power":np.power(x, 1/np.cbrt(50))}
sns.lineplot(data)